In [1]:
%load_ext autoreload
%autoreload 2

---

# SQLAlchemy

In [1]:
import sqlalchemy as sql

In [2]:
sql.__version__

'2.0.31'

In [3]:
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///:memory:", echo=True)

In [29]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [30]:
from sqlalchemy import Table, Column, Integer, String, DateTime, Interval
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
    Column('date', DateTime),
    Column('delta', Interval),
)

In [31]:
user_table.c.keys()

['id', 'name', 'fullname', 'date', 'delta']

In [9]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [9]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [27]:
metadata_obj.drop_all(engine)

2024-07-08 14:04:53,056 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 14:04:53,057 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-07-08 14:04:53,057 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-08 14:04:53,059 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2024-07-08 14:04:53,059 INFO sqlalchemy.engine.Engine [no key 0.00060s] ()
2024-07-08 14:04:53,060 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
metadata_obj.create_all(engine)

2024-07-08 14:05:38,651 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 14:05:38,652 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-07-08 14:05:38,653 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-08 14:05:38,654 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-07-08 14:05:38,655 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-08 14:05:38,656 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	date DATETIME, 
	delta DATETIME, 
	PRIMARY KEY (id)
)


2024-07-08 14:05:38,656 INFO sqlalchemy.engine.Engine [no key 0.00041s] ()
2024-07-08 14:05:38,657 INFO sqlalchemy.engine.Engine COMMIT


In [46]:
import datetime

dt = datetime.datetime.now(tz=datetime.UTC)
dt

datetime.datetime(2024, 7, 8, 12, 7, 28, 13392, tzinfo=datetime.timezone.utc)

In [47]:
td = datetime.timedelta(hours=3)

In [48]:
from sqlalchemy import insert, select
stmt = insert(user_table).values(id=2, name="spongebob", fullname="Spongebob Squarepants", date=dt, delta=td)

In [49]:
print(stmt)

INSERT INTO user_account (id, name, fullname, date, delta) VALUES (:id, :name, :fullname, :date, :delta)


In [51]:
stmt = select(user_table.c.date, user_table.c.delta)

In [52]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2024-07-08 14:07:34,565 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 14:07:34,566 INFO sqlalchemy.engine.Engine SELECT user_account.date, user_account.delta 
FROM user_account
2024-07-08 14:07:34,567 INFO sqlalchemy.engine.Engine [cached since 19.24s ago] ()
2024-07-08 14:07:34,568 INFO sqlalchemy.engine.Engine COMMIT


In [53]:
res = list(result)
res

[(datetime.datetime(2024, 7, 8, 12, 7, 28, 13392), datetime.timedelta(seconds=10800))]

In [24]:
dt = res[0][0]

In [26]:
dt.tzinfo is None

True

In [15]:
result.inserted_primary_key

(2,)

In [23]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {'id': 3, "name": "sandy", "fullname": "Sandy Cheeks"},
            {'id': 4, "name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2024-07-04 12:47:19,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:47:19,016 INFO sqlalchemy.engine.Engine INSERT INTO user_account (id, name, fullname) VALUES (?, ?, ?)
2024-07-04 12:47:19,017 INFO sqlalchemy.engine.Engine [cached since 15.82s ago] [(3, 'sandy', 'Sandy Cheeks'), (4, 'patrick', 'Patrick Star')]
2024-07-04 12:47:19,018 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2024-07-04 12:54:30,951 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:54:30,952 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2024-07-04 12:54:30,953 INFO sqlalchemy.engine.Engine [generated in 0.00153s] [('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org')]
2024-07-04 12:54:30,954 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
stmt = select(user_table).where(user_table.c.name == "spongebob")

In [28]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row.id)

2024-07-04 13:06:33,303 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 13:06:33,304 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-07-04 13:06:33,305 INFO sqlalchemy.engine.Engine [cached since 30.57s ago] ('spongebob',)
2
2024-07-04 13:06:33,305 INFO sqlalchemy.engine.Engine ROLLBACK


In [29]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [30]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


In [31]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address
